In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Folder_For_Project"

In [ ]:
# Imports
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import gc
from IPython.display import display

# Sci-kit Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

# NN
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Conv2D, MaxPooling2D

In [ ]:
# Load file
df = pd.read_csv("filename.csv",
                #  usecols=['post_id', 'date', 'num_comments', 'score', 'total_awards',
                #           'compound_score', 'author_id', 'author_name']
                )

df

In [ ]:
# Get the X-variable columns
df_X = df[['x_col_names']].to_numpy()

# Get the Y-variable columns
df_Y = df[['y_col_names']].to_numpy()

# Clear up memory
del df
gc.collect()

# Get 80% of the data by date
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.2,
                                                    shuffle=False)

# Print
print(X_train)
print(X_test)
print(y_train)
print(y_test)

In [ ]:
# Normalize data
scaler = MinMaxScaler()

# Change depending on number of features
n_features = 5

print(scaler.fit(X_train))
xtrain_scale=scaler.transform(X_train)
#xtrain_scale = xtrain_scale.reshape(xtrain_scale.shape[0], xtrain_scale.shape[1], n_features)
xtrain_scale = xtrain_scale.reshape(xtrain_scale.shape[0], 4, n_features)

print(scaler.fit(X_test))
xval_scale=scaler.transform(X_test)
#xval_scale = xval_scale.reshape(xval_scale.shape[0], xval_scale.shape[1], n_features)
xval_scale = xval_scale.reshape(xval_scale.shape[0], 4, n_features)

print(scaler.fit(y_train))
ytrain_scale=scaler.transform(y_train)

print(scaler.fit(y_test))
yval_scale=scaler.transform(y_test)

# Set input and output dimensions
indim = xtrain_scale.shape[1]
outdim = ytrain_scale.shape[1]

# Clear existing models
tf.keras.backend.clear_session()

# Initialize LSTM
model = Sequential()

# First LSTM layer with Dropout regularisation
model.add(LSTM(units=50, return_sequences=False, input_shape=(indim, n_features)))
model.add(Dropout(0.2))

# Second LSTM layer
#model.add(LSTM(units=50, return_sequences=False))
#model.add(Dropout(0.2))

# Third LSTM layer
#model.add(LSTM(units=50, return_sequences=True))
#model.add(Dropout(0.1))

# Fourth LSTM layer
#model.add(LSTM(units=50))
#model.add(Dropout(0.5))

# Output layer
model.add(Dense(outdim))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(xtrain_scale, ytrain_scale, epochs=10, batch_size=128, verbose=0)

# Predict
ypred = model.predict(xval_scale)

# Scale back to original scale
ypred = scaler.inverse_transform(ypred)

In [ ]:
# Evaluate Accuracy Score

# Convert y_test and ypred to type ints
y_test = y_test.astype(int)
ypred = ypred.astype(int)

# Print overall model average
print("The overall accuracy score is:", accuracy_score(y_test, ypred))
print("The overall f1 score for is:", f1_score(y_test, ypred))